In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA Minimization

In this notebook, we verify our minimization algorithm using a concrete example.
We manually construct a DFA, visualize it, run the minimization, and visualize the result.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";

import {
    DFA,
    DFAState,
    State,
    TransRelDet
} from "./01-NFA-2-DFA";
import { minimize } from "./07-Minimize";
import { dfa2dot, dfa2string, renderLegend } from "./FSM-2-Dot";

const viz = await instance();

### Defining the DFA

We define the DFA $A = (Q, \Sigma, \delta, q_0, F)$ manually.

**Implementation Details:**
To ensure correct behavior with `RecursiveSet`, we pre-create the singleton sets representing our states ($\{0\}, \{1\}, \dots$). This allows us to cache them and ensures referential identity where useful, though v7 relies on structural equality.

**Transitions:**
* $(0, a) \to 1, \quad (0, b) \to 6$
* ... (see code for full definition)

In [ ]:
const states = Array.from({ length: 9 }, (_, i) => new RecursiveSet(i));
const transitionData: [number, string, number][] = [
    [0, "a", 1], [0, "b", 6],
    [1, "a", 2], [1, "b", 7],
    [2, "a", 3], [2, "b", 6],
    [3, "a", 0], [3, "b", 7],
    [4, "a", 5], [4, "b", 2],
    [5, "a", 6], [5, "b", 8],
    [6, "a", 7], [6, "b", 2],
    [7, "a", 0], [7, "b", 3],
    [8, "a", 4], [8, "b", 8]
];
const δ = new RecursiveMap<Tuple<[DFAState, string]>, DFAState>();
for (const [from, char, to] of transitionData)
    δ.set(new Tuple(states[from], char), states[to]);
const A: DFA = { Q: new RecursiveSet(...states), Σ: new RecursiveSet("a", "b"), δ: δ, q0: states[0], A: new RecursiveSet(states[2], states[3], states[6], states[7])};

In [ ]:
dfa2string(A);

### Visualize Original DFA

We generate both the text representation (to see the state encoding) and the Graphviz diagram.

**Note:** The state names (S0, S1...) in the graph are generated automatically. Check the "state encoding" in the text output to map them back to our sets (e.g., see which S-name corresponds to `{0}`).

In [ ]:
const dot = dfa2dot(A);
display.html(viz.renderString(dot, { format: "svg" }));

### Minimize and Visualize

We apply the `minimize` function to compute the **Quotient Automaton**.

**Mathematical Structure:**
The result `F` is a `MinDFA`.
* Structurally, this matches our generic `DFA` interface, so all our helper functions continue to work.
* Semantically, each state in `F.Q` is an **Equivalence Class** (a set containing the original states that are indistinguishable).

For example, since states `2` and `6` are equivalent in the original DFA, they are merged into a single state in the minimized DFA:
$$S_{new} = \{ \{2\}, \{6\} \}$$

In [ ]:
const F = minimize(A);

In [ ]:
dfa2string(F);

### Visualize Minimized DFA

Since our visualization tools (`dfa2dot`, `renderLegend`) are defined generically for any state type that implements value equality, we can visualize the minimized DFA immediately.

* The **Nodes** in the graph now represent sets of original states.
* The **Legend** below shows exactly which original states were merged into which new equivalence class.

In [ ]:
const dot2 = dfa2dot(F);
display.html(viz.renderString(dot2, { format: "svg" }));

In [ ]:
display.html(renderLegend(F))